# Tutorial: using OpenMRS to recommend music for a listener

## Prerequisites

Please make sure you have installed the following packages:
```
numpy
sklearn
```

### Import OpenMRS library

In [1]:
try:
    import OpenMRS as om
except:
    # At this point, you probably haven't installed OpenMRS. You can install it by:
    #   sudo pip install git+https://github.com/BerryAI/Acai
    # Now we are going to import OpenMRS from the source.
    # Note: This assumes you are currently in the 'examples/' folder running this notebook.
    import os
    import sys
    CWD = os.getcwd()
    print 'current working directory:', CWD
    sys.path.append(os.path.join(CWD, '..'))
    import OpenMRS as om

current working directory: /Users/zz/projects/berryai/Acai/examples


### Load example data

OpenMRS provides a small set of user ratings data. Let's load it and see what's in there.

In [2]:
example_ratings = om.data.get_example_ratings()
example_tracks = om.data.get_example_tracks()

print 'These are user ids:'
print example_ratings.keys()[:5]

print 'These are track ids and rating scores for the first user:'
print example_ratings.values()[0].items()[:5]

These are user ids:
[u'5733935958982656', u'5693048138760192', u'6350668029755392', u'5640238160412672', u'4613861369970688']
These are track ids and rating scores for the first user:
[(u'2fuNBnTkGdi46PEsityDN6', 4), (u'1vG4lS2fKu6ahMLBXNAy2Z', 2), (u'5pPKsIik5j7pJ17NESh2ZE', 2), (u'3cY2jV291FokhXKnDTtgAn', 1), (u'4iCn374fZg5UbGD1xFNo1m', 2)]


### Start and train a recomemndation engine

Feed the user ratings data into the recommendation engine and train it with collaborative filtering.

In [3]:
engine = om.RecommendationEngine()  # or equivalently, use the following line
# engine = om.RecommendationEngine(catalog=SimpleCatalog(example_tracks))

# Feed the user ratings into the recommendation engine and train it.
engine.train(ratings=example_ratings)

Now the recommendation `engine` knows about users and their ratings. We can retrieve and print them out.

In [4]:
one_user = engine.get_user_ids()[0]
ratings = engine.get_ratings_by_user(user_id=one_user)
print ('Ratings by user %s (5 being most favorable and 1 least favorable):' %
    one_user)
for track_id, rating in ratings.items()[:5]:
    print '  User rates %s on track %s' % (rating,
        engine.catalog.get_track_by_id(track_id))


Ratings by user 5733935958982656 (5 being most favorable and 1 least favorable):
  User rates 4 on track 2fuNBnTkGdi46PEsityDN6: "Elegy for strings, Op.58", by Edward Elgar et al
  User rates 2 on track 1vG4lS2fKu6ahMLBXNAy2Z: "The Love for 3 Oranges Suite, Op. 3...", by Jascha Heifetz et al
  User rates 2 on track 5pPKsIik5j7pJ17NESh2ZE: "Song for Baba (Arr. Chowhan)", by Julian Lloyd Webber et al
  User rates 1 on track 3cY2jV291FokhXKnDTtgAn: "Girls Were Made To Love And Kiss", by André Rieu
  User rates 2 on track 4iCn374fZg5UbGD1xFNo1m: "Symphony No. 3: II. Lento E Largo -...", by Henryk Górecki


### Generate recommendations

With the trained `engine`, we can let it generate music recommendations for a user according to his/her historical ratings.

In [5]:
# Recommend tracks for a user.
recommended_tracks = engine.recommend(user_id=one_user, num=10)
print '\nRecommended tracks for user %s:' % one_user
for t in recommended_tracks:
    print t



Recommended tracks for user 5733935958982656:
5Bqc0WFB9d8Uf9pdNml6M3: "Kheira's Theme", by Julian Lloyd Webber
6f3ki4Xfbiv1iR33XCXgpJ: "Piano Trio No. 1, in D Minor, Op. 4...", by Pablo Casals et al
4XcOOsBx4InAFNzhKumQuv: "Songbird", by All Angels
0na8x6EOqjHsGABdB90TIC: "Hold Tight", by Justin Bieber
6cAglVR27Jy5asrhDVrCio: "Fields Of Fortune", by Rolf Løvland et al
0pOAcehnnODZXk37z39nOw: "Silent O Moyle", by Meav
5E3WulVOsri3J9MZpJ8QMZ: "Sonata for Violin and Piano in B-Fl...", by Arthur Grumiaux et al
7nUeYb7hCkOqfMBoMB8dly: "Waltz Of The White Lilies", by Deanta
5pPKsIik5j7pJ17NESh2ZE: "Song for Baba (Arr. Chowhan)", by Julian Lloyd Webber et al
66T30PDdNToSYj55OgteNG: "Suite for Solo Cello No. 2 in D Min...", by Sara Sant'Ambrogio


You can start streaming the recommended tracks to the user. Note that some tracks will be the same as what this user has listened to, so you if you want to encourage diversity, make sure to exclude the recently played tracks.

And after your users rate these recommended tracks, you can keep feeding the data back to the recommendation engine and get new recommendations.